In [1]:
import pandas as pd
import numpy as np 
import matplotlib as plt 
import seaborn as sns 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import json
import nltk

In [15]:
#import sklearn
#print(sklearn.__file__)

c:\Users\zelal\anaconda3\lib\site-packages\sklearn\__init__.py


In [2]:
df = pd.read_excel("C://Users/zelal/OneDrive/Desktop/Class Folders/CIS 630 - Project/Datasets_withTokens.xlsx")

In [3]:

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zelal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

sw_nltk = stopwords.words('english')
print(sw_nltk)
stop_words = set(stopwords.words('english'))
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
output_file_path = 'C://Users/zelal/OneDrive/Desktop/Class Folders/CIS 630 - Project/Dataset2.json'

In [6]:
with open(output_file_path, 'r') as json_file:
    data2 = json.load(json_file)

In [8]:
data2

{'intents': [{'tag': 'accounts',
   'patterns': ['What',
    'are',
    'the',
    'key',
    'features',
    'of',
    'Payment',
    'gateway',
    'of',
    'HDFC',
    'bank'],
   'responses': ['- Acceptance of online payments through cards/ NetBanking and IMPS - Value Added Services such as EMI (Easy Monthly Instalment), Standing Instructions (for recurring payment), IVR (Interactive Voice Response System), intelligent online reporting of succes']},
  {'tag': 'accounts',
   'patterns': ['How',
    'much',
    'of',
    'foreign',
    'exchange',
    'earnings',
    'can',
    'be',
    'credited',
    'into',
    'an',
    'EEFC',
    'account'],
   'responses': ["100% foreign exchange earnings can be credited to the EEFC account subject to the condition that the sum total of the accruals in the account during a calendar month should be converted into rupees on or before the last day of the succeeding calendar month after adjusting for utilisation of the balances for approved purp

In [7]:
# now that this worked, moving on to training and testing the data and model
# the variable "training_sentences" hold all the training data 
# the "training_labels" variable holds all the target lables corresponding to each training data

training_sentences = []
training_labels = []
labels = []
responses = []

In [8]:
# iterate through each intent in the JSON data
for intent in data2['intents']:
    # iterate through each pattern in the 'patterns' list of the intent
    for pattern in intent['patterns']:
        # tokenize the pattern into words
        words = word_tokenize(pattern)
        # remove the stop words from the tokenized words list
        words = [word for word in words if word.lower() not in stop_words]
        # join the remaining words to form the cleaned pattern
        cleaned_pattern = ' '.join(words)
        # append the cleaned pattern to the training sentences list
        training_sentences.append(cleaned_pattern)
        # append the intent tag to the training labels list
        training_labels.append(intent['tag'])
    
    # append the intent's responses to the responses list    
    responses.append(intent['responses'])
    
    # if the intent's tag is not already in the labels list, append it 
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [9]:
# then we use the "LableEncoder()" function provided by scikit-learn to convert lables into a model understandable form
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [10]:
# next we vectorize the data using the 'Tokenizer' class

vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
#word_index - tokenizer.word_index
squences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(squences, truncating='post', maxlen=max_len)

In [11]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 7)                 119       
                                                                 
Total params: 16,663
Trainable params: 16,663
Non-trainable params: 0
____________________________________________________

In [12]:
epochs = 50
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/50
457/457 [==============================] - 1s 1ms/step - loss: 1.7925 - accuracy: 0.2570
Epoch 2/50
457/457 [==============================] - 0s 985us/step - loss: 1.7331 - accuracy: 0.2568
Epoch 3/50
457/457 [==============================] - 0s 956us/step - loss: 1.7243 - accuracy: 0.2589
Epoch 4/50
457/457 [==============================] - 1s 1ms/step - loss: 1.6678 - accuracy: 0.2942
Epoch 5/50
457/457 [==============================] - 0s 975us/step - loss: 1.6042 - accuracy: 0.3152
Epoch 6/50
457/457 [==============================] - 0s 1ms/step - loss: 1.5779 - accuracy: 0.3278
Epoch 7/50
457/457 [==============================] - 0s 966us/step - loss: 1.5659 - accuracy: 0.3422
Epoch 8/50
457/457 [==============================] - 0s 985us/step - loss: 1.5529 - accuracy: 0.3620
Epoch 9/50
457/457 [==============================] - 0s 953us/step - loss: 1.5345 - accuracy: 0.3735
Epoch 10/50
457/457 [==============================] - 0s 968us/step - loss: 1.5070 - ac

In [13]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets


INFO:tensorflow:Assets written to: chat_model\assets


In [1]:
""" import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open(output_file_path, 'r') as json_file:
    data2 = json.load(json_file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data2['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , i['responses'][0])

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat() """

' import json \nimport numpy as np\nfrom tensorflow import keras\nfrom sklearn.preprocessing import LabelEncoder\n\nimport colorama \ncolorama.init()\nfrom colorama import Fore, Style, Back\n\nimport random\nimport pickle\n\nwith open(output_file_path, \'r\') as json_file:\n    data2 = json.load(json_file)\n\n\ndef chat():\n    # load trained model\n    model = keras.models.load_model(\'chat_model\')\n\n    # load tokenizer object\n    with open(\'tokenizer.pickle\', \'rb\') as handle:\n        tokenizer = pickle.load(handle)\n\n    # load label encoder object\n    with open(\'label_encoder.pickle\', \'rb\') as enc:\n        lbl_encoder = pickle.load(enc)\n\n    # parameters\n    max_len = 20\n    \n    while True:\n        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")\n        inp = input()\n        if inp.lower() == "quit":\n            break\n\n        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),\n            

In [14]:
# this code will reduce the answers to just one response
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open(output_file_path, 'r') as json_file:
    data2 = json.load(json_file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data2['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , random.choice(i['responses']))
                break

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()


Start messaging with the bot (type quit to stop)!
1/1 [==============================] - 0s 68ms/step
ChatBot: (Loans Against Rental Receivables)We request you to visit our nearest Retail Asset Branch along with a written request for a duplicate No Objection Certificate (NOC) and your photo identity proof.
User: 